# Gibt es „Angstgegner“-Nationen, gegen die die deutschen Spieler besonders schlecht abschneiden? Lässt sich das einfach durch die generelle Spielstärke der Spieler dieser Nationen erklären, oder gibt es Nationen, gegen die gerade deutsche Spieler schlecht abschneiden? Gibt es analog „Lieblingsgegner“-Nationen?

In [28]:
import json
import pandas as pd
import plotly.express as px

In [29]:
matches = pd.read_csv('../../data/raw/atp_matches_till_2022.csv')
players = pd.read_csv('../../data/raw/atp_players_till_2022.csv')

In [30]:
with open('../../data/processed/länder.json', 'r') as f:
    data = json.load(f)

codes = list(data.keys())
names = list(data.values())

länder = pd.DataFrame({'Code': codes, 'Name': names})

In [31]:
german_ioc_code = "GER"
german_winner_matches = matches[matches["winner_ioc"] == german_ioc_code]
german_loser_matches = matches[matches["loser_ioc"] == german_ioc_code]

siege = german_winner_matches["loser_ioc"].value_counts()
niederlagen = german_loser_matches["winner_ioc"].value_counts()

performance = pd.DataFrame({"Siege": siege, "Niederlagen": niederlagen}).fillna(0)

performance["Total"] = performance.sum(axis=1)
performance["Win_Rate"] = performance["Siege"] / performance["Total"]
performance["Loss_Rate"] = performance["Niederlagen"] / performance["Total"]

Grenze = 20

performance = performance[performance["Total"] >= Grenze].sort_values(by="Win_Rate", ascending=False)
performance = pd.concat([performance, länder.set_index('Code')], axis=1, join='inner').reset_index()

top = 10
top_favoriten = performance.head(top)
top_rivalen = performance.tail(top)

In [32]:
performance.sort_values(by="Win_Rate", ascending=True).head(10)

,index,Siege,Niederlagen,Total,Win_Rate,Loss_Rate,Name
58,CYP,12.0,36.0,48.0,0.250000,0.750000,Zypern
57,UZB,11.0,31.0,42.0,0.261905,0.738095,Usbekistan
56,LTU,7.0,15.0,22.0,0.318182,0.681818,Litauen
55,MAR,25.0,43.0,68.0,0.367647,0.632353,Marokko
54,ZIM,18.0,30.0,48.0,0.375000,0.625000,Simbabwe
53,PAR,17.0,26.0,43.0,0.395349,0.604651,Paraguay
52,BUL,14.0,21.0,35.0,0.400000,0.600000,Bulgarien
51,SUI,169.0,245.0,414.0,0.408213,0.591787,Schweiz
50,CHI,87.0,122.0,209.0,0.416268,0.583732,Chile
49,SRB,95.0,133.0,228.0,0.416667,0.583333,Serbien


In [33]:
print(top_favoriten)
print()
print(top_rivalen.sort_values(by="Loss_Rate",ascending=False))

  index  Siege  Niederlagen  Total  Win_Rate  Loss_Rate          Name
0   VEN   20.0          3.0   23.0  0.869565   0.130435     Venezuela
1   TPE   24.0          7.0   31.0  0.774194   0.225806        Taiwan
2   THA   23.0         10.0   33.0  0.696970   0.303030      Thailand
3   SLO   24.0         12.0   36.0  0.666667   0.333333     Slowenien
4   DEN   55.0         33.0   88.0  0.625000   0.375000      Dänemark
5   NOR   20.0         12.0   32.0  0.625000   0.375000      Norwegen
6   MEX   41.0         25.0   66.0  0.621212   0.378788        Mexiko
7   POR   31.0         19.0   50.0  0.620000   0.380000      Portugal
8   YUG   13.0          8.0   21.0  0.619048   0.380952   Jugoslawien
9   BLR   32.0         21.0   53.0  0.603774   0.396226  Weißrussland

   index  Siege  Niederlagen  Total  Win_Rate  Loss_Rate        Name
58   CYP   12.0         36.0   48.0  0.250000   0.750000      Zypern
57   UZB   11.0         31.0   42.0  0.261905   0.738095  Usbekistan
56   LTU    7.0       

In [34]:
fig = px.bar(performance.sort_values('Win_Rate', ascending=False), 
            y=['Win_Rate'], 
            x='index',
            title='Gewinnrate der deutschen Tennisspieler gegen andere Nationen',
            hover_data=['Siege', 'Niederlagen', 'Name'],
            color_discrete_sequence= ["#b2e061", "#fd7f6f"],
            opacity=0.9)

fig.add_hline(y=0.5, opacity=0.4, line_dash='dot', line_color='black')

fig.add_annotation(text="50% Gewinnrate",
                xref='paper',
                y=performance['Win_Rate'].mean(),
                arrowhead=1, 
                showarrow=True, 
                arrowcolor='black',        
                bgcolor='white',
                font=dict(size=10, color='grey'))

fig.add_annotation(text="Lieblingsgegner",
                   xref='paper',
                   y= performance['Win_Rate'].median(),
                   x=0.05,
                   showarrow=False, 
                   bgcolor='white',
                   font=dict(size=10, color='grey'))

fig.add_annotation(text="Angstgegner",
                   xref='paper',
                   y=performance['Win_Rate'].median(),
                   x=0.95, 
                   showarrow=False, 
                   bgcolor='white',
                   font=dict(size=10, color='grey'))

fig.update_yaxes(tickformat=".2%")
fig.update_yaxes(title='Gewinnrate')
fig.update_xaxes(title='Gegner Nationen')
fig.show()


## Liegen die teilweise geringen Siegesquoten der Deutschan an der generellen Spielstärke der Gegnernation?

In [35]:
#Dataframe der Gewinnernationen
df_loser = matches.groupby(["loser_ioc", "winner_ioc"]).size().reset_index(name ="loses_count")

#Dataframe der Verlierernationen
df_winner = matches.groupby(["winner_ioc", "loser_ioc"]).size().reset_index(name="wins_count")

In [36]:
#Summe der Siege pro Nation
df_winner = df_winner.groupby("winner_ioc")["wins_count"].sum().reset_index()
#Summe der Niederlagen pro Nation 
df_loser = df_loser.groupby("loser_ioc")["loses_count"].sum().reset_index()

In [37]:
# Berechnung der Anzahl der gewonnenen Spiele pro Land
wins = matches['winner_ioc'].value_counts().reset_index()
wins.columns = ['nation', 'wins']

# Berechnung der Anzahl der verlorenen Spiele pro Land
losses = matches['loser_ioc'].value_counts().reset_index()
losses.columns = ['nation', 'losses']

# Zusammenführen der DataFrames
df_winrate = pd.merge(wins, losses, on='nation', how='outer')

# Füllen der fehlenden Werte mit 0
df_winrate = df_winrate.fillna(0)

# Berechnung der Gesamtzahl der Spiele
df_winrate['total_games'] = df_winrate['wins'] + df_winrate['losses']

# Berechnung der Gewinnrate
df_winrate['win_rate'] = df_winrate['wins'] / df_winrate['total_games']


## Prüfen, ob die geringe Siegesquote Deutschlands an der generellen Spielspärke der Gegnernation liegt

In [38]:
#Tabellen joinen, um generelle Sigesquote der Gegenernationen der Sigesquote von Deutschland gegen diese Nation gegenüberzustellen
df_merge3 = pd.merge(performance, df_winrate[['nation', 'win_rate', 'wins', 'losses']], how="outer", left_on="index", right_on="nation")
df_merge3.rename(columns={"win_rate": "win_rate_Gegner_allgemein"}, inplace=True)
df_merge3.rename(columns={'Win_Rate': 'win_rate_Deutschland'}, inplace=True)

In [39]:
df_merge3

,index,Siege,Niederlagen,Total,win_rate_Deutschland,Loss_Rate,Name,nation,win_rate_Gegner_allgemein,wins,losses
0,VEN,20.0,3.0,23.0,0.869565,0.130435,Venezuela,VEN,0.376828,335.0,554.0
1,TPE,24.0,7.0,31.0,0.774194,0.225806,Taiwan,TPE,0.406286,349.0,510.0
2,THA,23.0,10.0,33.0,0.696970,0.303030,Thailand,THA,0.508399,454.0,439.0
3,SLO,24.0,12.0,36.0,0.666667,0.333333,Slowenien,SLO,0.454787,342.0,410.0
4,DEN,55.0,33.0,88.0,0.625000,0.375000,Dänemark,DEN,0.445946,726.0,902.0
...,...,...,...,...,...,...,...,...,...,...,...
137,NaN,NaN,NaN,NaN,NaN,NaN,NaN,BER,0.000000,0.0,2.0
138,NaN,NaN,NaN,NaN,NaN,NaN,NaN,FIJ,0.000000,0.0,1.0
139,NaN,NaN,NaN,NaN,NaN,NaN,NaN,AZE,0.000000,0.0,1.0
140,NaN,NaN,NaN,NaN,NaN,NaN,NaN,SUR,0.000000,0.0,1.0


In [40]:
fig = px.bar(df_merge3, 
            y=['win_rate_Gegner_allgemein', 'win_rate_Deutschland'], 
            x='index',
            title='Gewinnrate der deutschen Tennisspieler gegen andere Nationen',
            barmode="group", 
            color_discrete_sequence= ["#fd7f6f", "#b2e061"],
            opacity=0.9,
            hover_data=['Name'])

fig.add_hline(y=0.5, opacity=0.4, line_dash='dot', line_color='black')

fig.add_annotation(text="50% Gewinnrate",
                xref='paper',
                y=performance['Win_Rate'].mean(),
                arrowhead=1, 
                showarrow=True, 
                arrowcolor='black',        
                bgcolor='white',
                font=dict(size=10, color='grey'))

fig.update_yaxes(tickformat=".2%")
fig.update_yaxes(title='Gewinnrate in %')
fig.update_xaxes(title='Gegner Nationen')
fig.show()

## Korrelation

In [41]:
korrelation = df_merge3["win_rate_Deutschland"].corr(df_merge3["win_rate_Gegner_allgemein"])
korrelation

-0.44872103872407154

Das bedeutet, dass die Siegesquote von Deutschland gegen die jeweilige Gegneration sinkt, wenn die allgemeine Siegesquote der Gegnernation steigt.

Der Power Bi Report für Aufgabe 1 ist an den Trainer der deutschen Tennisspieler gerichtet.